In [1]:
import db.database as db
import helpers.document_utils as utils

conn = db.db_connection()
products = db.get_all_products(conn)
attributes = db.get_all_attributes(conn)

print("---Sample product---")
for k, v in products[0].items():
    print(f"{k}: {v}")

conn.close()

---Sample product---
id: 1
uid: 8Hjz-PQh4UF0m-laF8
product_type: 102
seller_id: 1
warranty_policy: None
brand_id: 1
category_id: 1
sub_category_id: 2
name: Samsung Galaxy S25 Ultra 5G ( 12/256 12/512 12/1Tera ) Garansi Resmi
slug: samsung-galaxy-s25-ultra
price: 17549000.00000000
shipping_fee: 20000.0
shipping_fee_multiply: 0
discount: None
discount_percentage: 0E-8
featured_image: 681a169ef27251746540190.jpg
short_description: <!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.0 Transitional//EN" "http://www.w3.org/TR/REC-html40/loose.dtd">
<html><head><meta http-equiv="Content-Type" content="text/html; charset=utf-8">
<meta http-equiv="Content-Type" content="text/html; charset=utf-8"></head><body><span style="color: rgb(8, 8, 8); font-family: " open sauce one sans-serif font-size:>Sambut era baru Smartphone dengan AI yang selangkah lebih maju untuk setiap kebutuhan Anda. Biarkan AI membantu Anda untuk menyelesaikan tugas sehari-hari dengan mudah.</span><br style="box-sizing: inherit; color: r

In [2]:
documents = utils.generate_product_documents(products, attributes)
print("---Sample product document---")
print(documents[4].page_content)

---Sample product document---
**Product Name**: Kemeja Unisex Erigo Short Shirt Jazlyn Black - Jazlyn Black

**Description**: Erigo Short Shirt Rayon kemeja pendek berbahan rayon kami dirancang untuk memberikan sentuhan yang lembut dan ringan di kulit, kemeja ini menjadi pilihan ideal untuk tampil kasual yang tetap terlihat rapi.
Rayon memiliki sifat elastisitas yang memungkinkan fleksibilitas gerakan yang lebih baik, menjadikan kemeja ini pilihan yang tepat untuk kegiatan sehari-hari. Tersedia dalam berbagai pilihan warna yang menarik, memungkinkan untuk mengekspresikan gaya pribadi dengan lebih bebas.
Reverensi Size
Untuk Model Pria: Tinggi 185-186 cm, Berat 75 kg, Menggunakan Ukuran XL
Untuk Model Wanita: Tinggi 168-170 cm, Berat 55 kg, Menggunakan Ukuran M
Bahan: Katun Rayon
PERHATIAN
Kain bahan rayon merupakan jenis kain yang menyusut setelah dicuci. Disarankan untuk memilih size 1 tingkat lebih besar dari size biasanya.
Toleransi setiap size 1-1cm

Product "Kemeja Unisex Erigo Sh

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-m3")

texts = [f"Passage: {doc.page_content}" for doc in documents]

for i, text in enumerate(texts):
    tokens = tokenizer.encode(text, truncation=False)
    # print(f"Doc {i} → {len(tokens)} tokens")

    if len(tokens) > 8194:
        print(f"⚠️  Exceeds token limit! Token: ", len(tokens))

In [4]:
from sentence_transformers import SentenceTransformer

# from langchain.text_splitter import RecursiveCharacterTextSplitter

# splitter = RecursiveCharacterTextSplitter(
#     chunk_size=1024, 
#     chunk_overlap=100,
#     strip_whitespace=True,
#     separators=["\n\n", "\n", ".", " ", ""],
#     keep_separator=False
# )
# chunks = splitter.split_documents(documents)

texts = [f"Passage: {doc.page_content}" for doc in documents]

print(f"Sample chunk: {texts[0][:200]}...")

model = SentenceTransformer('BAAI/bge-m3')

embeddings = model.encode(
    texts,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True  # for cosine similarity
)

print(f"Embeddings shape: {embeddings.shape}")
print(f"Embedding dimension: {embeddings.shape[1]}")

Sample chunk: Passage: **Product Name**: Samsung Galaxy S25 Ultra 5G ( 12/256 12/512 12/1Tera ) Garansi Resmi

**Description**: NOTE: Khusus bonus Watch series yang dapat FREE akan dikirim random sesuai stock yang ...


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Embeddings shape: (104, 1024)
Embedding dimension: 1024


In [5]:
import faiss

dimension = embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)

index.add(embeddings)
print(f"FAISS Index total vectors: {index.ntotal}")

FAISS Index total vectors: 104


In [6]:
# Save FAISS index file to reuse later
import pickle

faiss.write_index(index, "data/tokopoin_product.index")

with open("data/chunk_texts.pkl", "wb") as f:
    pickle.dump(texts, f)

In [7]:
import faiss
index = faiss.read_index('./data/tokopoin_product.index')

print("-----FAISS Vector Store Statistics-----\n")

# Jumlah vektor (jumlah dokumen yang di-indeks)
print("FAISS Index total vectors:", index.ntotal)

# Dimensi dari tiap vektor
print("Vector Dimension: ", index.d)

# Jenis index yang digunakan
print("Index type: ", type(index))

-----FAISS Vector Store Statistics-----

FAISS Index total vectors: 104
Vector Dimension:  1024
Index type:  <class 'faiss.swigfaiss_avx2.IndexFlatIP'>
